In [1]:


import pandas as pd
from pandas import Series,DataFrame

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

import csv
import gzip

from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import cross_validation
from sklearn.ensemble.forest import RandomForestRegressor

In [66]:
Train_GD = pd.read_csv("CAX_Bidding_TRAIN_Molecule_3_4_5.csv")
Pre_Test_GD = pd.read_csv("CAX_Bidding_TEST_Molecule_6_Pre_LOE.csv")
Pos_Test_GD = pd.read_csv("CAX_Bidding_TEST_Molecule_6_Post_LOE.csv")
Sample_Submission = pd.read_csv("CAX_Bidding_SubmissionFormat.csv")
Dict_GD = pd.read_excel("CAX_Bidding_DataDictionary.xlsx")
Sub_Train_df = pd.read_csv("Subsetted Data.csv")
Sub_Test_df = pd.read_csv("Subsetted Data_Test.csv")


In [67]:
Sub_Test_df.shape

(152, 16)

In [68]:
Sub_Train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 786 entries, 0 to 785
Data columns (total 16 columns):
ID                                 786 non-null int64
Account                            786 non-null object
Province                           786 non-null object
Region                             786 non-null object
Tender_Type                        786 non-null object
Offer_date                         786 non-null object
Start_date                         786 non-null object
End_date_including_extension       786 non-null object
Length_of_contract_in_Months       786 non-null int64
Market                             786 non-null object
Tender_Standard_Unit_per_year      786 non-null float64
Winning_price_per_standard_unit    745 non-null float64
Molecule                           786 non-null object
Presentation                       786 non-null object
Winning_Product                    747 non-null object
Winning_Company                    747 non-null object
dtypes: float64

In [69]:
#Splitting the date column
def create_month(x):
    return x.split('/')[0]

def create_day(x):
    return x.split('/')[1]

def create_year(x):
    return x.split('/')[2]

In [70]:
Sub_Train_df["Date_of_Offer"] = Sub_Train_df["Offer_date"].apply(create_day)
Sub_Train_df["Month_of_Offer"] = Sub_Train_df["Offer_date"].apply(create_month)
Sub_Train_df["Year_of_Offer"] = Sub_Train_df["Offer_date"].apply(create_year)

Sub_Test_df["Date_of_Offer"] = Sub_Test_df["Offer_date"].apply(create_day)
Sub_Test_df["Month_of_Offer"] = Sub_Test_df["Offer_date"].apply(create_month)
Sub_Test_df["Year_of_Offer"] = Sub_Test_df["Offer_date"].apply(create_year)


In [71]:
Sub_Train_df.head()

,ID,Account,Province,Region,Tender_Type,Offer_date,Start_date,End_date_including_extension,Length_of_contract_in_Months,Market,Tender_Standard_Unit_per_year,Winning_price_per_standard_unit,Molecule,Presentation,Winning_Product,Winning_Company,Date_of_Offer,Month_of_Offer,Year_of_Offer
0,226,Account95,Provice21,Region6,Tender Type1,10/3/2008 12:00,1/1/2009,31-12-2014,36,Market3,69673,1.56353,Molecule3,Presentation8,Product10,Company1,3,10,2008 12:00
1,227,Account95,Provice21,Region6,Tender Type1,4/13/2010 12:00,1/5/2010,30-09-2013,20,Market3,56966,1.56353,Molecule3,Presentation8,Product10,Company1,13,4,2010 12:00
2,232,Account111,Provice15,Region13,Tender Type1,4/12/2010 12:00,1/5/2010,30-06-2012,24,Market3,211912,1.77000,Molecule3,Presentation8,Product10,Company1,12,4,2010 12:00
3,240,Account91,Provice10,Region9,Tender Type1,5/10/2010 0:00,1/7/2010,30-09-2012,24,Market3,26050,1.77000,Molecule3,Presentation8,Product10,Company1,10,5,2010 0:00
4,242,Account51,Provice32,Region7,Tender Type1,5/19/2010 12:00,1/8/2010,31-05-2012,12,Market3,192786,1.50000,Molecule3,Presentation8,Product10,Company1,19,5,2010 12:00


In [72]:
Sub_Train_df["Year_of_Offer"] = Sub_Train_df["Year_of_Offer"].apply(lambda x: int(str(x)[:5]))
Sub_Test_df["Year_of_Offer"] = Sub_Test_df["Year_of_Offer"].apply(lambda x: int(str(x)[:5]))


In [73]:
#dropping those rows which has missing values
Sub_Train_df.dropna(subset=['Winning_Product'], inplace=True)
Sub_Test_df.dropna(subset=['Winning_Product'], inplace=True)
Sub_Train_df.dropna(subset=["Winning_price_per_standard_unit"], inplace=True)

In [74]:
Sub_Train_df.info()
print "-------------------------"
Sub_Test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745 entries, 0 to 785
Data columns (total 19 columns):
ID                                 745 non-null int64
Account                            745 non-null object
Province                           745 non-null object
Region                             745 non-null object
Tender_Type                        745 non-null object
Offer_date                         745 non-null object
Start_date                         745 non-null object
End_date_including_extension       745 non-null object
Length_of_contract_in_Months       745 non-null int64
Market                             745 non-null object
Tender_Standard_Unit_per_year      745 non-null float64
Winning_price_per_standard_unit    745 non-null float64
Molecule                           745 non-null object
Presentation                       745 non-null object
Winning_Product                    745 non-null object
Winning_Company                    745 non-null object
Date_of_Offer  

In [75]:
# Convert categorical data to numbers
from sklearn import ensemble, preprocessing
lbl = preprocessing.LabelEncoder()
lbl.fit(list(Sub_Train_df['Account'].values) + list(Sub_Test_df['Account'].values))
Sub_Train_df['Account'] = lbl.transform(Sub_Train_df['Account'].values)
Sub_Test_df['Account'] = lbl.transform(Sub_Test_df['Account'].values)

lbl.fit(list(Sub_Train_df['Province'].values) + list(Sub_Test_df['Province'].values))
Sub_Train_df['Province'] = lbl.transform(Sub_Train_df['Province'].values)
Sub_Test_df['Province'] = lbl.transform(Sub_Test_df['Province'].values)

lbl.fit(list(Sub_Train_df['Region'].values) + list(Sub_Test_df['Region'].values))
Sub_Train_df['Region'] = lbl.transform(Sub_Train_df['Region'].values)
Sub_Test_df['Region'] = lbl.transform(Sub_Test_df['Region'].values)

lbl.fit(list(Sub_Train_df['Account'].values) + list(Sub_Test_df['Account'].values))
Sub_Train_df['Account'] = lbl.transform(Sub_Train_df['Account'].values)
Sub_Test_df['Account'] = lbl.transform(Sub_Test_df['Account'].values)

lbl.fit(list(Sub_Train_df['Tender_Type'].values) + list(Sub_Test_df['Tender_Type'].values))
Sub_Train_df['Tender_Type'] = lbl.transform(Sub_Train_df['Tender_Type'].values)
Sub_Test_df['Tender_Type'] = lbl.transform(Sub_Test_df['Tender_Type'].values)

lbl.fit(list(Sub_Train_df['Presentation'].values) + list(Sub_Test_df['Presentation'].values))
Sub_Train_df['Presentation'] = lbl.transform(Sub_Train_df['Presentation'].values)
Sub_Test_df['Presentation'] = lbl.transform(Sub_Test_df['Presentation'].values)

lbl.fit(list(Sub_Train_df['Winning_Product'].values) + list(Sub_Test_df['Winning_Product'].values))
Sub_Train_df['Winning_Product'] = lbl.transform(Sub_Train_df['Winning_Product'].values)
Sub_Test_df['Winning_Product'] = lbl.transform(Sub_Test_df['Winning_Product'].values)

lbl.fit(list(Sub_Train_df['Winning_Company'].values) + list(Sub_Test_df['Winning_Company'].values))
Sub_Train_df['Winning_Company'] = lbl.transform(Sub_Train_df['Winning_Company'].values)
Sub_Test_df['Winning_Company'] = lbl.transform(Sub_Test_df['Winning_Company'].values)

lbl.fit(list(Sub_Train_df['Market'].values) + list(Sub_Test_df['Market'].values))
Sub_Train_df['Market'] = lbl.transform(Sub_Train_df['Market'].values)
Sub_Test_df['Market'] = lbl.transform(Sub_Test_df['Market'].values)


In [76]:
#Dropping unwanted columns
Sub_Train_df.drop(['Offer_date','Start_date','End_date_including_extension',
                   'Molecule'], axis = 1, inplace=True)
Sub_Test_df.drop(['Offer_date','Start_date','End_date_including_extension',
                  'Molecule'], axis = 1, inplace=True)


In [77]:
#Sub_Train_df.dropna(subset=["Winning_price_per_standard_unit"], inplace=True)

In [78]:
Sub_Train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745 entries, 0 to 785
Data columns (total 15 columns):
ID                                 745 non-null int64
Account                            745 non-null int64
Province                           745 non-null int64
Region                             745 non-null int64
Tender_Type                        745 non-null int64
Length_of_contract_in_Months       745 non-null int64
Market                             745 non-null int64
Tender_Standard_Unit_per_year      745 non-null float64
Winning_price_per_standard_unit    745 non-null float64
Presentation                       745 non-null int64
Winning_Product                    745 non-null int64
Winning_Company                    745 non-null int64
Date_of_Offer                      745 non-null object
Month_of_Offer                     745 non-null object
Year_of_Offer                      745 non-null int64
dtypes: float64(2), int64(11), object(2)
memory usage: 93.1+ KB


In [79]:
Sub_Test_df.head(2)

,ID,Account,Province,Region,Tender_Type,Length_of_contract_in_Months,Market,Tender_Standard_Unit_per_year,Winning_price_per_standard_unit,Presentation,Winning_Product,Winning_Company,Date_of_Offer,Month_of_Offer,Year_of_Offer
0,2543,51,11,11,1,4,3,28800,NaN,10,72,17,28,2,2011
1,2544,3,22,13,0,36,3,6500,NaN,10,72,17,14,2,2011


In [80]:
Sub_Train_df.drop("ID",axis=1,inplace=True)
Sub_Test_df.drop(["Winning_price_per_standard_unit"],axis=1,inplace=True)

In [81]:
#Training and testing data
X_train = Sub_Train_df.drop("Winning_price_per_standard_unit",axis=1)
Y_train = Sub_Train_df["Winning_price_per_standard_unit"]
X_test = Sub_Test_df.drop("ID",axis=1)

In [82]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745 entries, 0 to 785
Data columns (total 13 columns):
Account                          745 non-null int64
Province                         745 non-null int64
Region                           745 non-null int64
Tender_Type                      745 non-null int64
Length_of_contract_in_Months     745 non-null int64
Market                           745 non-null int64
Tender_Standard_Unit_per_year    745 non-null float64
Presentation                     745 non-null int64
Winning_Product                  745 non-null int64
Winning_Company                  745 non-null int64
Date_of_Offer                    745 non-null object
Month_of_Offer                   745 non-null object
Year_of_Offer                    745 non-null int64
dtypes: float64(1), int64(10), object(2)
memory usage: 81.5+ KB


In [83]:
# Linear Regression
from sklearn.linear_model import LinearRegression
lreg = LinearRegression()
clf = lreg.fit(X_train, Y_train)
Y_pred = lreg.predict(X_test)
lreg.score(X_train, Y_train)

0.41479177817909407

In [85]:
#Random forest regressor
rf = RandomForestRegressor(n_jobs = -1, n_estimators = 15)
rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)
rf.score(X_train,Y_train)

0.96237120150906763

In [86]:
Submission = pd.DataFrame({
        "ID": Sub_Test_df["ID"],
        "Winning_price_per_standard_unit": Y_pred
        
    })
Submission.to_csv('RandomR.csv', index=False)